In [14]:
text = open('2023_24.txt', 'r').read()

text1 = '''19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3'''

data = []
for line in text.split('\n'):
    if line:
        data.append([tuple(map(int, x.split(', '))) for x in line.split(' @ ')])
   

def line_rectangle_intersect(m, b, left, right, top, bottom):
    # Calculate intersection points with rectangle's boundaries
    x_at_top = (top - b) / m
    x_at_bottom = (bottom - b) / m
    y_at_left = m * left + b
    y_at_right = m * right + b
    # Check if intersection points are within rectangle's bounds
    if left <= x_at_top <= right or left <= x_at_bottom <= right or bottom <= y_at_left <= top or bottom <= y_at_right <= top:
        return True
    return False

if len(data) > 10:
    minx = 200000000000000
    maxx = 400000000000000
    miny = 200000000000000
    maxy = 400000000000000
else:
    minx = 7
    maxx = 27
    miny = 7
    maxy = 27  
solcount=0

for p1ind in range(len(data)):
    p1 = data[p1ind]
    # t = 0 is on the line, (x,y) = (p1[0][0],p1[0][1])
    # t = 1 is on the line, (x,y) = (p1[0][0]+p1[1][0],p1[0][1]+p1[1][1])
    # y = m1*x + b1
    m1 = p1[1][1]/p1[1][0]
    b1 = -m1*p1[0][0] + p1[0][1]
    if line_rectangle_intersect(m1, b1, miny, maxy, maxx, minx):
        for p2ind in range(p1ind+1, len(data)):
            p2 = data[p2ind]
            if p1 != p2:        
                m2 = p2[1][1]/p2[1][0]
                b2 = -m2*p2[0][0] + p2[0][1]
                if m1 == m2:
                    continue
                x_val = (b2 - b1) / (m1 - m2)
                y_val = m1 * x_val + b1
                if (x_val-p1[0][0])/p1[1][0] > 0 and (x_val-p2[0][0])/p2[1][0] > 0:
                    if minx <= x_val <= maxx and miny <= y_val <= maxy:
                        solcount += 1
                        #print(p1ind, p2ind, x_val, y_val,(x_val-p1[0][0])/p1[1][0])
                        #print((x_val-p1[0][0])/p1[1][0], (y_val-p1[0][1])/p1[1][1])
                        #print(solcount)

            
print('Part 1:',solcount)

Part 1: 24192


In [38]:
# Part 2
import numpy as np
from sympy import symbols, Eq, solve
# create a new data set with the first point subtracted from all points
data2 = []
for p in data:
    data2.append([(p[0][0]-data[0][0][0],p[0][1]-data[0][0][1],p[0][2]-data[0][0][2]),(p[1][0]-data[0][1][0],p[1][1]-data[0][1][1],p[1][2]-data[0][1][2])])

p1 = (0,0,0)
p2 = data2[1][0]
p3 = (data2[1][0][0]+data2[1][1][0],data2[1][0][1]+data2[1][1][1],data2[1][0][2]+data2[1][1][2])
print(p1,p2,p3)
# find where the line described by data2[2][0]+t*data2[2][1] intesects the plane described by p1,p2,p3

# Plane equation coefficients (Ax + By + Cz + D = 0)
plane_normal = np.cross(np.subtract(p2, p1), np.subtract(p3, p1))
A, B, C = plane_normal
D = -np.dot(plane_normal, p1)

# Line equation
line_point = np.array(data2[2][0])
line_dir = np.array(data2[2][1])

# Parameter t for the line
t = symbols('t')
x, y, z = line_point + t * line_dir

# Intersection equation
intersection_eq = Eq(A * x + B * y + C * z + D, 0)

# Solving for t
t_value = solve(intersection_eq, t)

# Printing the result
print(f"Intersection at t = {t_value}")
            
# point of intersection
p_int = (data2[2][0][0]+t_value[0]*data2[2][1][0],data2[2][0][1]+t_value[0]*data2[2][1][1],data2[2][0][2]+t_value[0]*data2[2][1][2])
print(p_int)

# the rock will hit p_int at time t_value[0]

# a*p_int = data2[1][0] + c*data2[1][1]
a, c = symbols('a c')
eq1 = a*p_int[0] - data2[1][0][0] - c*data2[1][1][0]
eq2 = a*p_int[1] - data2[1][0][1] - c*data2[1][1][1]
eq3 = a*p_int[2] - data2[1][0][2] - c*data2[1][1][2]
solution = solve((eq1, eq2, eq3), (a, c))
print(solution)

# the rock a*p_int at time c 
tdiff = solution[c] - t_value[0]

a=solution[a]

dx = (a-1)*p_int[0]/tdiff
dy = (a-1)*p_int[1]/tdiff
dz = (a-1)*p_int[2]/tdiff

x= p_int[0] - dx*t_value[0]
y= p_int[1] - dy*t_value[0]
z= p_int[2] - dz*t_value[0]

print(x,y,z)


print('Part 2:',x+data[0][0][0]+y+data[0][0][1]+z+data[0][0][2])

(0, 0, 0) (-57865360466310, -150705600824138, -37609536098360) (-57865360466038, -150705600823808, -37609536097885)
Intersection at t = [857431055888]
(173240884569196, 124397639580524, 370140215929780)
{a: 358087827407/381587851474, c: 810431007754}
-21395965117380 -15363622529220 -45713846175900
Part 2: 664822352550558
